### Парсинг на основе MARK Down

In [4]:
import os
from pathlib import Path
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.oxml import OxmlElement

def docx_to_markdown(docx_path, output_path=None):
    """
    Конвертирует .docx файл в markdown формат.
    
    Args:
        docx_path: путь к .docx файлу
        output_path: путь для сохранения markdown файла (опционально)
    
    Returns:
        str: содержимое документа в markdown формате
    """
    doc = Document(docx_path)
    markdown_lines = []
    
    def get_paragraph_style_level(paragraph):
        """Определяет уровень заголовка по стилю параграфа"""
        style_name = paragraph.style.name.lower()
        if 'heading' in style_name:
            try:
                level = int(style_name.replace('heading', '').strip())
                return min(level, 6)  # Markdown поддерживает только 6 уровней
            except:
                return 1
        return None
    
    def process_run(run):
        """Обрабатывает run (фрагмент текста) с форматированием"""
        text = run.text
        if not text:
            return ""
        
        # Жирный текст
        if run.bold:
            text = f"**{text}**"
        
        # Курсив
        if run.italic:
            text = f"*{text}*"
        
        # Подчеркивание (в markdown нет нативного подчеркивания, используем <u>)
        if run.underline:
            text = f"<u>{text}</u>"
        
        # Зачеркнутый текст
        if run.font.strike or (hasattr(run, '_element') and run._element.get(qn('w:strike'))):
            text = f"~~{text}~~"
        
        return text
    
    def process_paragraph(paragraph):
        """Обрабатывает параграф"""
        # Пропускаем пустые параграфы
        if not paragraph.text.strip() and not paragraph.runs:
            return ""
        
        # Проверяем, является ли параграф заголовком
        heading_level = get_paragraph_style_level(paragraph)
        if heading_level:
            text = paragraph.text.strip()
            if text:
                markdown_lines.append(f"{'#' * heading_level} {text}\n")
            return
        
        # Обрабатываем текст параграфа с форматированием
        paragraph_text = ""
        for run in paragraph.runs:
            paragraph_text += process_run(run)
        
        # Проверяем, является ли параграф частью списка
        if paragraph.style.name.startswith('List'):
            # Определяем тип списка (нумерованный или маркированный)
            if 'number' in paragraph.style.name.lower() or 'numbered' in paragraph.style.name.lower():
                # Для нумерованных списков можно использовать автоматическую нумерацию
                markdown_lines.append(f"1. {paragraph_text.strip()}\n")
            else:
                markdown_lines.append(f"- {paragraph_text.strip()}\n")
        else:
            if paragraph_text.strip():
                markdown_lines.append(f"{paragraph_text.strip()}\n")
    
    def process_table(table):
        """Обрабатывает таблицу"""
        if not table.rows:
            return
        
        # Заголовок таблицы (первая строка)
        header_row = table.rows[0]
        headers = [cell.text.strip() for cell in header_row.cells]
        markdown_lines.append("| " + " | ".join(headers) + " |\n")
        markdown_lines.append("| " + " | ".join(["---"] * len(headers)) + " |\n")
        
        # Остальные строки
        for row in table.rows[1:]:
            cells = [cell.text.strip().replace('\n', ' ') for cell in row.cells]
            # Дополняем ячейки, если их меньше, чем заголовков
            while len(cells) < len(headers):
                cells.append("")
            markdown_lines.append("| " + " | ".join(cells) + " |\n")
        
        markdown_lines.append("\n")
    
    # Обрабатываем все элементы документа
    for element in doc.element.body:
        if element.tag.endswith('p'):  # Параграф
            paragraph = None
            for p in doc.paragraphs:
                if p._element == element:
                    paragraph = p
                    break
            if paragraph:
                process_paragraph(paragraph)
        elif element.tag.endswith('tbl'):  # Таблица
            table = None
            for t in doc.tables:
                if t._element == element:
                    table = t
                    break
            if table:
                process_table(table)
    
    # Объединяем все строки
    markdown_content = "".join(markdown_lines)
    
    # Сохраняем в файл, если указан путь
    if output_path:
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(markdown_content)
        print(f"Markdown файл сохранен: {output_path}")
    
    return markdown_content

# Пример использования
if __name__ == "__main__":
    # Тестируем на одном из файлов
    test_file = "/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/1 кейс Сценарии c Таблицами/Фишер (с таблицей)/1/ФИШЕР 1 сери __16.05.docx"
    if os.path.exists(test_file):
        markdown_text = docx_to_markdown(test_file)
        print("Первые 1000 символов markdown:")
        print(markdown_text[:1000])
    else:
        print(f"Файл не найден: {test_file}")


Первые 1000 символов markdown:
|  |
| --- |
| ФИШЕР |
| первая серия |
|  |
|  |

|  |
| --- |
| Авторы сценария Наталья КАПУСТИНА Сергей КАЛЬВАРСКИЙ при участии Михаила БАРКАНа |

# Серия 1
**1-1. НАТ. ЛЕС.ПОЛЯНА НОЧЬ 1. **
**ИГОРЬ, ПАНОВ****, труп Андрея Панова, собака Лесси**** (01:55)**
Иногда в Подмосковье бывают такие ночи, когда в небе показывают картину Ван Гога про Млечный путь. Это именно она. Сказочная, невероятная. Взгляд опускается с небес на землю, и мы видим милого мальчика с биноклем, наблюдающего за звёздами. Он гладит собаку, рисует предполагаемый путь кометы в блокноте, в свете фонарика, он выглядит как маленький принц, изучающий планеты. Старательно что-то записывает в дневнике.
ИГОРЬ (ЗК)
Мне только что исполнилось 10 лет. Это был первый день рождения, на который не пришёл никто из моих друзей. Потому что друзей у меня здесь нет.
Поворачивает морду собаки к дневнику.
ИГОРЬ
Я не буду писать, что ты мой друг, ладно?
Целует собаку. Пишет дальше.
ИГОРЬ
Мы только что ве

In [5]:
# Улучшенная версия парсера с поддержкой списков и правильным порядком элементов

import os
import re
from docx import Document
from docx.oxml.ns import qn

def docx_to_markdown_advanced(docx_path, output_path=None):
    """
    Улучшенная версия парсера .docx в markdown.
    Поддерживает:
    - Заголовки всех уровней (H1-H6)
    - Форматирование текста (жирный, курсив, подчеркивание, зачеркнутый)
    - Нумерованные и маркированные списки (с поддержкой вложенности)
    - Таблицы
    - Гиперссылки
    - Правильный порядок элементов (параграфы и таблицы в нужной последовательности)
    """
    doc = Document(docx_path)
    markdown_lines = []
    
    # Создаем словари для быстрого доступа к параграфам и таблицам по элементам
    paragraphs_dict = {p._element: p for p in doc.paragraphs}
    tables_dict = {t._element: t for t in doc.tables}
    
    def get_heading_level(paragraph):
        """Определяет уровень заголовка по стилю"""
        style_name = paragraph.style.name.lower()
        if 'heading' in style_name or 'заголовок' in style_name:
            # Извлекаем номер из названия стиля
            for word in style_name.split():
                if word.isdigit():
                    level = int(word)
                    return min(max(level, 1), 6)
            # Попытка определить по стандартным названиям
            if 'heading 1' in style_name or 'заголовок 1' in style_name:
                return 1
            elif 'heading 2' in style_name or 'заголовок 2' in style_name:
                return 2
            elif 'heading 3' in style_name or 'заголовок 3' in style_name:
                return 3
            elif 'heading 4' in style_name or 'заголовок 4' in style_name:
                return 4
            elif 'heading 5' in style_name or 'заголовок 5' in style_name:
                return 5
            elif 'heading 6' in style_name or 'заголовок 6' in style_name:
                return 6
        return None
    
    def is_list_paragraph(paragraph):
        """Проверяет, является ли параграф элементом списка через XML"""
        p_element = paragraph._element
        # Проверяем наличие нумерации или маркировки
        numPr = p_element.find('.//{http://schemas.openxmlformats.org/wordprocessingml/2006/main}numPr')
        return numPr is not None
    
    def get_list_info(paragraph):
        """Получает информацию о списке (тип и уровень)"""
        p_element = paragraph._element
        numPr = p_element.find('.//{http://schemas.openxmlformats.org/wordprocessingml/2006/main}numPr')
        
        if numPr is None:
            return None, 0
        
        # Определяем уровень вложенности
        ilvl = numPr.find('.//{http://schemas.openxmlformats.org/wordprocessingml/2006/main}ilvl')
        level = 0
        if ilvl is not None and ilvl.get(qn('w:val')) is not None:
            try:
                level = int(ilvl.get(qn('w:val')))
            except:
                pass
        
        # Определяем тип списка (нумерованный или маркированный)
        numId = numPr.find('.//{http://schemas.openxmlformats.org/wordprocessingml/2006/main}numId')
        is_numbered = True  # По умолчанию нумерованный
        
        if numId is not None:
            num_id_val = numId.get(qn('w:val'))
            # Проверяем определение списка в документе
            # В реальных документах нужно проверять numbering.xml, но для простоты используем эвристику
            style_name = paragraph.style.name.lower()
            if 'bullet' in style_name or 'маркер' in style_name:
                is_numbered = False
        
        return is_numbered, level
    
    def process_run(run):
        """Обрабатывает run (фрагмент текста) с форматированием"""
        text = run.text
        if not text:
            return ""
        
        # Обработка гиперссылок
        try:
            hyperlinks = run._element.xpath('.//w:hyperlink', 
                namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'})
            if hyperlinks:
                for hyperlink in hyperlinks:
                    rId = hyperlink.get(qn('r:id'))
                    if rId and rId in doc.part.rels:
                        rel = doc.part.rels[rId]
                        url = rel.target_ref
                        # Извлекаем текст ссылки из элемента
                        link_texts = []
                        for t in hyperlink.iter():
                            if t.text:
                                link_texts.append(t.text)
                        link_text = ''.join(link_texts)
                        if link_text:
                            text = text.replace(link_text, f"[{link_text}]({url})", 1)
        except:
            pass  # Если не удалось обработать ссылку, продолжаем
        
        # Жирный текст
        if run.bold:
            text = f"**{text}**"
        
        # Курсив
        if run.italic:
            text = f"*{text}*"
        
        # Подчеркивание
        if run.underline:
            text = f"<u>{text}</u>"
        
        # Зачеркнутый текст
        try:
            strike_elem = run._element.find(
                './/{http://schemas.openxmlformats.org/wordprocessingml/2006/main}strike')
            if strike_elem is not None:
                text = f"~~{text}~~"
        except:
            pass
        
        return text
    
    def process_paragraph(paragraph):
        """Обрабатывает параграф"""
        # Собираем текст из всех runs с форматированием
        text_content = ""
        for run in paragraph.runs:
            text_content += process_run(run)
        
        text_content = text_content.strip()
        
        # Пропускаем пустые параграфы (кроме случаев, когда это важно для структуры)
        if not text_content and not paragraph.runs:
            return
        
        # Проверяем заголовки
        heading_level = get_heading_level(paragraph)
        if heading_level:
            if text_content:
                markdown_lines.append(f"{'#' * heading_level} {text_content}\n\n")
            return
        
        # Проверяем, является ли параграф элементом списка
        list_info = get_list_info(paragraph)
        is_list_item, list_level = list_info
        
        if is_list_item is not None:
            # Это элемент списка
            indent = "  " * list_level
            if is_list_item:
                markdown_lines.append(f"{indent}1. {text_content}\n")
            else:
                markdown_lines.append(f"{indent}- {text_content}\n")
        else:
            # Обычный параграф
            if text_content:
                markdown_lines.append(f"{text_content}\n\n")
            else:
                markdown_lines.append("\n")
    
    def process_table(table):
        """Обрабатывает таблицу"""
        if not table.rows:
            return
        
        markdown_lines.append("\n")  # Отступ перед таблицей
        
        # Обрабатываем каждую строку
        for i, row in enumerate(table.rows):
            cells = []
            for cell in row.cells:
                # Обрабатываем содержимое ячейки (могут быть параграфы)
                cell_texts = []
                for para in cell.paragraphs:
                    para_text = ""
                    for run in para.runs:
                        para_text += process_run(run)
                    if para_text.strip():
                        cell_texts.append(para_text.strip())
                cell_text = " ".join(cell_texts) if cell_texts else " "
                cell_text = cell_text.replace('\n', ' ').strip()
                cells.append(cell_text if cell_text else " ")
            
            if cells:
                markdown_lines.append("| " + " | ".join(cells) + " |\n")
                
                # Добавляем разделитель после первой строки
                if i == 0:
                    markdown_lines.append("| " + " | ".join(["---"] * len(cells)) + " |\n")
        
        markdown_lines.append("\n")  # Отступ после таблицы
    
    # Обрабатываем элементы в правильном порядке (как они идут в документе)
    for element in doc.element.body:
        if element.tag.endswith('p'):  # Параграф
            paragraph = paragraphs_dict.get(element)
            if paragraph:
                process_paragraph(paragraph)
        elif element.tag.endswith('tbl'):  # Таблица
            table = tables_dict.get(element)
            if table:
                process_table(table)
    
    # Объединяем результат
    markdown_content = "".join(markdown_lines)
    
    # Очищаем лишние пустые строки (более 2 подряд)
    markdown_content = re.sub(r'\n{3,}', '\n\n', markdown_content)
    
    # Убираем лишние пробелы в начале и конце
    markdown_content = markdown_content.strip() + '\n'
    
    # Сохраняем файл
    if output_path:
        output_dir = os.path.dirname(output_path)
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(markdown_content)
        print(f"✓ Markdown файл сохранен: {output_path}")
    
    return markdown_content


In [7]:
# Тестирование парсера на реальных файлах

import os
from pathlib import Path

# Путь к папке с данными
data_dir = Path("../../../data/1 кейс Сценарии c Таблицами")

# Находим первый .docx файл для тестирования
docx_files = list(data_dir.rglob("*.docx"))

if docx_files:
    test_file = docx_files[0]
    print(f"Тестируем файл: {test_file.name}")
    print("-" * 50)
    
    # Конвертируем в markdown
    markdown_result = docx_to_markdown_advanced(str(test_file))
    
    # Показываем первые 2000 символов
    print("Результат конвертации (первые 2000 символов):")
    print("=" * 50)
    print(markdown_result[:2000])
    print("=" * 50)
    print(f"\nОбщая длина markdown: {len(markdown_result)} символов")
    
    # Сохраняем результат
    output_file = f"output_{test_file.stem}.md"
    docx_to_markdown_advanced(str(test_file), output_file)
else:
    print("Не найдено .docx файлов для тестирования")


Тестируем файл: ЧЕЛЮСКИН_1с_15.08_ФИНАЛ.docx
--------------------------------------------------
Результат конвертации (первые 2000 символов):
**1.1 НАТ. КЛАДБИЩЕ КИТОВ ****–**** ВЕЧЕР****. ****СД 1. 00:15**

ВЫКВЫН

На берегу ледовитого океана торчат белыми ребрами вверх остовы китов. Посреди них - одинокая фигура - чукча-шаман ВЫКВЫН (46). Над ним яркими, переливающимися красками раскинулось полярное сияние. Выквын смотрит вдаль, на север - то ли ждет чего-то или кого-то, то ли беззвучно зовет, то ли молится.

**1.2A НАТ. ВАНКАРЕМ. БЕРЕГ/ ЛЕД - ВЕЧЕР - (УДАЛЕНА)**

**1.2 НАТ. ЛЁД. ВМЕРЗШИЙ "ЧЕЛЮСКИН" - ВЕЧЕР****. ****СД 1. 00:10**

Такое же яркое сияние раскинулось над зажатым льдами пароходом с надписью «Челюскин» на борту. С правого борта на лед ведет трап, у трапа - нахоженные тропинки, в иллюминаторах горит свет.

**1-3-N1. ИНТ. ЧЕЛЮСКИН. КАЮТ-КОМПАНИЯ. НОЧЬ****. ****СД 1. 00:50**

АЛЛОЧКА, МАМА АЛЛОЧКИ, ПАПА АЛЛОЧКИ, ТЕВОСОВ, ТАРАСОВ, СОМОВА, ШМИДТ, ГОРСКАЯ, ПЕТЬКА-МЕХАНИК, ПРОХО

In [9]:
with open('/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/ЧЕЛЮСКИН_1с_15.08_ФИНАЛ.md', 'w', encoding='utf-8') as f:
    f.write(markdown_result)

In [10]:
from tqdm.auto import tqdm

/Users/egorbykov/Desktop/Работа/python/python_venvs/Hacatons_3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Функция для массовой обработки всех .docx файлов

def batch_convert_docx_to_markdown(input_dir, output_dir=None, recursive=True):
    """
    Конвертирует все .docx файлы в директории в markdown формат.
    
    Args:
        input_dir: путь к директории с .docx файлами
        output_dir: путь к директории для сохранения .md файлов (если None, создается рядом)
        recursive: искать файлы рекурсивно
    
    Returns:
        list: список путей к созданным markdown файлам
    """
    input_path = Path(input_dir)
    
    if output_dir is None:
        output_path = input_path / "markdown_output"
    else:
        output_path = Path(output_dir)
    
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Находим все .docx файлы
    if recursive:
        docx_files = list(input_path.rglob("*.docx"))
    else:
        docx_files = list(input_path.glob("*.docx"))
    
    converted_files = []
    
    for docx_file in tqdm(docx_files):
        try:
            # Создаем относительный путь для сохранения структуры директорий
            relative_path = docx_file.relative_to(input_path)
            md_file_name = relative_path.with_suffix('.md')
            md_file_path = output_path / md_file_name
            
            # Создаем необходимые поддиректории
            md_file_path.parent.mkdir(parents=True, exist_ok=True)
            
            # Конвертируем файл
            print(f"Обрабатываю: {docx_file.name}...", end=" ")
            docx_to_markdown_advanced(str(docx_file), str(md_file_path))
            converted_files.append(md_file_path)
            print("✓")
        except Exception as e:
            print(f"✗ Ошибка при обработке {docx_file.name}: {e}")
    
    print(f"\nОбработано файлов: {len(converted_files)}/{len(docx_files)}")
    return converted_files

# Пример использования для массовой конвертации
# Раскомментируйте, чтобы запустить:
batch_convert_docx_to_markdown("../../../data", output_dir = '/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md',recursive=True)


  2%|▏         | 1/59 [00:00<00:10,  5.78it/s]

Обрабатываю: ЧЕЛЮСКИН_1с_15.08_ФИНАЛ.docx... ✓ Markdown файл сохранен: /Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_1с_15.08_ФИНАЛ.md
✓
Обрабатываю: ЧЕЛЮСКИН_6с_13.09_ФИНАЛ.docx... 

  5%|▌         | 3/59 [00:00<00:07,  7.34it/s]

✓ Markdown файл сохранен: /Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_6с_13.09_ФИНАЛ.md
✓
Обрабатываю: ЧЕЛЮСКИН_4с_10.09_ФИНАЛ.docx... ✓ Markdown файл сохранен: /Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_4с_10.09_ФИНАЛ.md
✓
Обрабатываю: ЧЕЛЮСКИН_5с_13.09_ФИНАЛ.docx... 

  8%|▊         | 5/59 [00:00<00:07,  7.54it/s]

...

 97%|█████████▋| 57/59 [00:04<00:00, 12.13it/s]

✓ Markdown файл сохранен: /Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Фишер (с таблицей)/2/ФИШЕР2серия8финальная2 28.11.24.md
✓
Обрабатываю: ФИШЕР2серия5финальная.docx... ✓ Markdown файл сохранен: /Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Фишер (с таблицей)/2/ФИШЕР2серия5финальная.md
✓
Обрабатываю: Фишер2серия1финальная.docx... ✓ Markdown файл сохранен: /Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Фишер (с таблицей)/2/Фишер2серия1финальная.md
✓
Обрабатываю: Фишер2серия6финальная.docx... 

100%|██████████| 59/59 [00:04<00:00, 11.90it/s]

✓ Markdown файл сохранен: /Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Фишер (с таблицей)/2/Фишер2серия6финальная.md
✓
Обрабатываю: Фишер2серия2финальная.docx... ✓ Markdown файл сохранен: /Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Фишер (с таблицей)/2/Фишер2серия2финальная.md
✓

Обработано файлов: 58/59


[PosixPath('/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_1с_15.08_ФИНАЛ.md'),
 PosixPath('/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_6с_13.09_ФИНАЛ.md'),
 PosixPath('/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_4с_10.09_ФИНАЛ.md'),
 PosixPath('/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_5с_13.09_ФИНАЛ.md'),
 PosixPath('/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_2С_15.08_ФИНАЛ.md'),
 PosixPath('/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Челюскин (с таблицей)/ЧЕЛЮСКИН_3С_05.09_ФИНАЛ.md'),
 PosixPath('/Users/egorbykov

## Использование парсера

### Базовое использование:
```python
from parsing.notebooks.hipotesis.parsing import docx_to_markdown_advanced

# Конвертация одного файла
markdown_text = docx_to_markdown_advanced("путь/к/файлу.docx")

# Конвертация с сохранением в файл
docx_to_markdown_advanced("путь/к/файлу.docx", "выходной_файл.md")
```

### Массовая обработка:
```python
# Обработка всех .docx файлов в директории
batch_convert_docx_to_markdown("../../../data", output_dir="output_markdown")
```

### Поддерживаемые элементы:
- ✅ Заголовки (H1-H6)
- ✅ Жирный и курсивный текст
- ✅ Подчеркивание и зачеркивание
- ✅ Нумерованные и маркированные списки
- ✅ Таблицы
- ✅ Гиперссылки
- ✅ Сохранение структуры документа


In [ ]:
# Пример быстрого использования парсера

# Конвертация одного файла
# markdown_text = docx_to_markdown_advanced("путь/к/файлу.docx", "выходной_файл.md")

# Или просто получить текст без сохранения
# markdown_text = docx_to_markdown_advanced("путь/к/файлу.docx")
# print(markdown_text)
